# Visualize a scattering event using my WebGL code

First, clone the GitHub repo: https://github.com/ageller/Scatter_WebGL

In [1]:
import os
import json
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

from IPython.display import IFrame

import http.server
import socketserver

from threading import Thread

In [2]:
# define the directory where the code lives (this can be anywhere on your computer; just change the directory below)
directory = os.path.join("/home/ynad93/tsunami/cusp_data/testing_files")

## Create some new data to replace the default data packaged with repo

For this example, I will read in some data in a csv file that I created a while ago.  (Note that this data is downsampled, and in some parts of the time sequence the trajectories do not look smooth.)

The data will need to be reformatted into a dict and output as a json with the following structure.  Each primary key is a time, and for each time there are particles that have the word "Particle" in the key name.  Each particle has at least a key of "r" with [x,y,z] values (anything else is ignored).

Below is a visual example of how the data should be formatted:

```
{
"0.1": {
  "Particle1": {
    "r" : [-44.59797843122882, -1.602469437175624, -6.039267952136012e-14],
   },
  "Particle2": {
    "r" : [21.90496690915433, 1.108629663443225, -0.01596151404716814],
   },
  "Particle3": {
    "r" : [22.6930115220745, 0.4938397737323992, 0.01596151404722853],
   }
},
"0.2": { ...
```

In [7]:
# read in the data
times = np.load(os.path.join(directory, "t.npy"))/(2*np.pi)
posall = np.load(os.path.join(directory, "pos.npy"))
starIDs = np.arange(posall.shape[1])

In [9]:
interp = interp1d(times, posall, kind='linear', axis=0)

In [10]:
#tlin = np.arange(times[0], times[-1], 1E-3)
tlin = np.linspace(times[0], times[-1], int((times[-1]-times[0])/1E-3))
f = interp(tlin)

In [16]:
where = np.where(np.mean(np.linalg.norm(f, axis=-1), axis=-1)<200)

In [17]:
where

(array([354499, 354500, 354502, ..., 420638, 420639, 420640]),)

In [22]:
f[0],f[-1]

(array([[ -131.24833751,  -270.43762333,   180.91977472],
        [ -131.22395685,  -270.54757403,   181.04227162],
        [ -129.5000651 ,  -273.23707974,   182.21088413],
        [   -4.68000118,  -434.60442039,   222.97729437],
        [  686.21139949,  1623.96845219, -1050.93375304],
        [  693.1035266 ,  1617.01436461, -1053.86444136]]),
 array([[   -27.74281677,    199.70811859,    442.51651552],
        [   -27.70526815,    199.69350322,    442.51022752],
        [  -508.43886638,  -9896.02954009, -14833.16972165],
        [   -31.6830949 ,    151.4977602 ,    435.8127425 ],
        [   726.04168318,   3171.55145606,   3014.85231992],
        [  -118.60597541,    259.87446931,    468.04115826]]))

In [18]:
# reformat and output to the json file
#Nframes = times.size
i0 = 354499
i1 = -1


newData = dict()
i = i0
posallfin = f
timesfin = tlin
for t in timesfin[i0:i1]:
    newData[t] = {}
    j = 0
    for iden in starIDs:
        newData[t]['Particle' + str(iden)] = {'r': [posallfin[i,j,0], posallfin[i,j,1], posallfin[i,j,2]]}
        j+=1
    i+=1

# serialize json
json_object = json.dumps(newData, indent = 4)
 
# write to file (ScatterParts.json is the file name, and it lives in the code's data directory)
# Note: this will replace the default data set
with open(os.path.join(os.getcwd(),'data','ScatterParts.json'), "w") as outfile:
    outfile.write(json_object)

In [14]:
f.shape

(644663, 6, 3)

In [19]:
# define the port that you want (8000 is standard)
port = 8002

In [20]:
# a simple class to start the http server in a thread (so that you still have access to the jupyter notebook)
class serverThread(Thread): 
    def run(self):
        handler = http.server.SimpleHTTPRequestHandler
        #os.chdir(directory)
        with socketserver.TCPServer(("", port), handler) as httpd:
            print("serving at port", port)
            httpd.serve_forever()

serverThread().start()

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/ynad93/miniconda3/envs/dany0/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/tmp/ipykernel_5868/252163901.py", line 6, in run
  File "/home/ynad93/miniconda3/envs/dany0/lib/python3.10/socketserver.py", line 452, in __init__
    self.server_bind()
  File "/home/ynad93/miniconda3/envs/dany0/lib/python3.10/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use


In [21]:
# create an iFrame to view the visualization in this notebook
IFrame("http://localhost:8002", width = 1200, height = 800)

127.0.0.1 - - [07/Feb/2023 10:04:11] "GET /data/ScatterParts.json HTTP/1.1" 200 -


In [2]:
import plotly.express as px
import plotly.graph_objs as go
import numpy as np

def animate_particles(positions):
    N, M, _ = positions.shape

    frames = [go.Frame(
        data=[go.Scatter3d(x=positions[i, :, 0], y=positions[i, :, 1], z=positions[i, :, 2],
                          mode='markers')],
        name=str(i)
    ) for i in range(N)]

    figure = go.Figure(
        data=[go.Scatter3d(x=positions[0, :, 0], y=positions[0, :, 1], z=positions[0, :, 2],
                         mode='markers')],
        frames=frames,
        layout=go.Layout(
            updatemenus=[dict(type='buttons',
                              showactive=False,
                              buttons=[dict(label='Play',
                                            method='animate',
                                            args=[None, dict(frame=dict(duration=50, redraw=False),
                                                             transition=dict(duration=0),
                                                             fromcurrent=True)])])]
        )
    )

    figure.show()

# Example usage:
N = 100  # number of time steps
M = 10   # number of particles
positions = np.random.rand(N, M, 3)
animate_particles(positions)
